In [1]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
#!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab
%cd /content/
# install detectron2:
!git clone https://github.com/facebookresearch/detectron2
%cd /content/detectron2
# !pip install -r requirements.txt
!python setup.py install
!pip install git+https://github.com/facebookresearch/fvcore.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-gb7jn1nt
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-gb7jn1nt
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
2.0.0+cu118 True
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

/content
fatal: destination path 'detectron2' already exists and is not an empty directory.
/content/detectron2
running install
/usr/local/lib/python3.9/dist-packages/setuptools/command

In [3]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
from torchvision.transforms.functional import to_tensor
from torchvision import transforms
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import torch.nn as nn
import torchvision
import torch
import detectron2
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog
from detectron2.data import transforms as T
from detectron2.structures import BoxMode
from detectron2.modeling import build_model
from PIL import Image, ImageDraw
from scipy.optimize import linear_sum_assignment
import numpy as np
import cv2


# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# # import some common libraries
# import numpy as np
# import cv2
# import random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [5]:
def xyxy_to_xywh(coordenadas):
    xmin, ymin, xmax, ymax = coordenadas
    x = xmin
    y = ymin
    w = xmax - xmin
    h = ymax - ymin
    return int(x), int(y), int(w), int(h)

def xywh_to_xyxy(coordenadas):
    x, y, w, h = coordenadas
    xmin = x
    ymin = y
    xmax = x + w
    ymax = y + h
    return int(xmin), int(ymin), int(xmax), int(ymax)

In [6]:
class PoolDetectionModel(nn.Module):
    def __init__(self, num_classes, num_predictions = False):
        super(PoolDetectionModel, self).__init__()
        
        # Cargamos el modelo pre-entrenado
        self.model = torch.hub.load('facebookresearch/detr:main', 'detr_resnet50', pretrained=True)

        # Cambiamos el número de clases de salida
        hidden_dim = self.model.transformer.d_model
        self.model.num_classes = num_classes
        self.model.class_embed = nn.Linear(hidden_dim, num_classes)
        
        if num_predictions != False:
          # Cambiamos el número predicciones
          num_query = self.model.transformer.d_model
          self.model.query_embed = nn.Embedding(num_predictions, num_query)
        
    def forward(self, x):
        # Pasamos las imágenes por el modelo pre-entrenado
        outputs = self.model(x)
        
        # Obtenemos las cajas y las etiquetas de las predicciones
        boxes = outputs['pred_boxes']
        labels = outputs['pred_logits'].sigmoid()
        
        # Devolvemos las cajas y las etiquetas
        return boxes, labels

In [7]:
def hungarian_loss_2(outputs, targets):
    boxes, labels = outputs
    true_boxes = torch.FloatTensor(targets['boxes']).to(device)
    true_labels = torch.FloatTensor(targets['labels']).to(device)
    true_labels = torch.FloatTensor([[label] for label in true_labels]).to(device)

    true_labels_0s = true_labels.flatten()
    # if paramaters["num_predictions"] > len(true_labels_0s):
    true_labels_0s = torch.cat((true_labels_0s, torch.zeros(paramaters["num_predictions"]-len(true_labels_0s)).to(device)), 0)

    # Calculamos la entropía cruzada binaria para todas las etiquetas
    all_label_loss = F.binary_cross_entropy(labels.flatten(), true_labels_0s)

    # Calculamos el costo de emparejamiento entre las cajas predichas y verdaderas
    cost_boxes = torch.cdist(boxes, true_boxes, p=1)
    
    # Calculamos el costo de emparejamiento entre las etiquetas predichas y verdaderas
    cost_labels = torch.cdist(labels, true_labels, p=1)
    
    # Combinamos los costos
    cost = cost_boxes + cost_labels
    
    # Resolvemos el problema de asignación lineal
    row_ind, col_ind = linear_sum_assignment(cost.cpu().detach().numpy())
    
    # Obtenemos las cajas y etiquetas predichas emparejadas
    boxes = boxes[row_ind]
    labels = labels[row_ind]
    
    # Obtenemos las cajas y etiquetas verdaderas emparejadas
    true_boxes = true_boxes[col_ind]
    true_labels = true_labels[col_ind]

    # Calculamos la pérdida de IoU para las cajas
    box_loss = F.mse_loss(boxes, true_boxes)
    
    # Calculamos la entropía cruzada binaria para las etiquetas emparejadas
    label_loss = F.binary_cross_entropy(labels, true_labels)
    

    # print(boxes[0])
    # print(labels[0][0])
    # Combinamos las pérdidas
    box_loss = box_loss
    label_loss = label_loss
    all_label_loss = all_label_loss
    loss = box_loss + label_loss + all_label_loss

    # print(loss.item(), box_loss.item(), label_loss.item(), all_label_loss.item())
    
    return loss

In [8]:
def hungarian_loss(outputs, targets):
    boxes, labels = outputs
    true_boxes, true_labels = torch.FloatTensor(targets['boxes']).to(device), torch.FloatTensor(targets['labels']).to(device)
    true_labels = torch.FloatTensor([[label] for label in true_labels]).to(device)

    # Calculamos el costo de emparejamiento entre las cajas predichas y verdaderas
    cost_boxes = torch.cdist(boxes, true_boxes, p=1)
    
    # Calculamos el costo de emparejamiento entre las etiquetas predichas y verdaderas
    cost_labels = torch.cdist(labels, true_labels, p=1)
    
    # Combinamos los costos
    cost = cost_boxes + cost_labels
    
    # Resolvemos el problema de asignación lineal
    row_ind, col_ind = linear_sum_assignment(cost.cpu().detach().numpy())
    
    # Obtenemos las cajas y etiquetas predichas emparejadas
    boxes = boxes[row_ind]
    labels = labels[row_ind]
    
    # Obtenemos las cajas y etiquetas verdaderas emparejadas
    true_boxes = true_boxes[col_ind]
    true_labels = true_labels[col_ind]

    # Calculamos la pérdida de IoU para las cajas
    box_loss = F.mse_loss(boxes, true_boxes)
    
    # Calculamos la entropía cruzada binaria para las etiquetas
    label_loss = F.binary_cross_entropy(labels, true_labels)
    
    # Combinamos las pérdidas
    loss = box_loss + label_loss
    
    return loss

In [9]:
def object_detection_loss(output, target):
    bbox_pred = torch.squeeze(output[0])
    prob_pred = torch.squeeze(output[1])

    bbox_true = torch.FloatTensor([t["boxes"][0] for t in target]).to(device)

    # prob_true = torch.FloatTensor([label for t in target for label in t["labels"]]).to(device)
    prob_true = torch.FloatTensor([t["labels"][0] for t in target]).to(device)


    # Calcula la pérdida de entropía cruzada con pesos para la clasificación
    classification_loss = F.binary_cross_entropy(prob_pred, prob_true)

    # Recorte provisional de las bbtrue
    

    # Calcula la pérdida de error cuadrático medio para la regresión
    regression_loss = F.mse_loss(bbox_pred, bbox_true)

    # Combina las pérdidas de clasificación y regresión
    loss = classification_loss + regression_loss

    return loss

In [10]:
def collate_fn(batch):
    images = []
    targets = []
    transform = transforms.Resize(new_size)
    # Define the normalization transform
    resize = transforms.Resize(new_size)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([transforms.ToTensor(), resize, normalize])
    for sample in batch:
        image = Image.open(sample['file_name']).convert('RGB')
        width, height = image.size
        # Transformar la imagen
        image = transform(image)
        # Adaptar las anotaciones
        annotations = sample['annotations']
        for ann in annotations:
            bbox = ann['bbox']
            x_original, y_original, w_original, h_original = bbox
            x_new = x_original / width #* new_size[1]
            y_new = y_original / height #* new_size[0]
            w_new = w_original / width #* new_size[1]
            h_new = h_original / height #* new_size[0]
            ann['bbox'] = [x_new, y_new, w_new, h_new]
            # print(bbox)
            # print(ann['bbox'])
        
        # Añadir la imagen y las anotaciones a la lista
        images.append(image)
        targets.append({'boxes': [ann['bbox'] for ann in annotations], 'labels': [1 for ann in annotations if ann['category_id'] == 0]})
        targets[-1]['boxes'].append([0, 0, 0, 0])
        targets[-1]['labels'].append(0)
    return images, targets

In [11]:
def collate_fn_test(batch):
    images = []
    targets = []
    transform = transforms.Resize(new_size)
    resize = transforms.Resize(new_size)
    transform = transforms.Compose([transforms.ToTensor(), resize])
    for sample in batch:
        image = Image.open(sample['file_name']).convert('RGB')
        width, height = image.size
        # Transformar la imagen
        image = transform(image)
        # Adaptar las anotaciones
        annotations = sample['annotations']
        for ann in annotations:
            bbox = ann['bbox']
            x_original, y_original, w_original, h_original = bbox
            x_new = x_original / width #* new_size[1]
            y_new = y_original / height #* new_size[0]
            w_new = w_original / width #* new_size[1]
            h_new = h_original / height #* new_size[0]
            ann['bbox'] = [x_new, y_new, w_new, h_new]
            # print(bbox)
            # print(ann['bbox'])
        
        # Añadir la imagen y las anotaciones a la lista
        images.append(image)
        targets.append({'boxes': [ann['bbox'] for ann in annotations], 'labels': [1 for ann in annotations if ann['category_id'] == 0]})
    return images, targets

In [12]:
def train(model, optimizer, data_loader, device, epochs=10):
    model.train()
    losses = []
    min_loss = 100000000000

    for name, param in model.named_parameters():
        if "query_embed" not in name and "bbox_embed" not in name:
            param.requires_grad = False
        else:
          print("No congelando:", name)
    for epoch in range(epochs):
      print("\nEpoch:", epoch, "\n")
      for images, targets in tqdm(data_loader):
          images = [image.to(device) for image in images]
          targets = [{k: v for k, v in t.items()} for t in targets]
          output = model(images)
          loss = 0
          output = [(prob, targ) for prob, targ in zip(output[0], output[1])]
          for o, t in zip(output, targets):
            loss += hungarian_loss_2(o, t)
            # print(o)
            # print(t)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          # losses = object_detection_loss(output, targets)
          loss = loss.item()/len(images)
          print(" - loss:", loss)
          losses.append(loss)
          # if min_loss > loss:
          #   min_loss = loss
          #   print("Save best model", loss)
          #   torch.save(model, f"{path_dataset}/model_best.pt")
          if len(losses) > 2 and abs(losses[-1] - losses[-2]) < 10**-6:
            return losses
      torch.save(model, f"{path_dataset}model.pt")
    return losses

In [ ]:
path_dataset = "/content/drive/MyDrive/TFG/dataset_700x700/"
paramaters = {"learning_rate": 0.0001,
              "momentum": 0.9,
              "batch_size": 4,
              "epochs": 100, 
              "num_classes": 1,
              "num_predictions": 100}

new_size = (700, 700)


DatasetCatalog.clear()
MetadataCatalog.clear()

# Declarar el modelo
model = PoolDetectionModel(num_classes=paramaters["num_classes"], num_predictions = paramaters["num_predictions"])

# Declarar el optimizador
optimizer = optim.SGD(model.parameters(), lr=paramaters["learning_rate"], momentum=paramaters["momentum"])

# Registrar instancia de COCO
register_coco_instances("piscinas_entrenamiento", {}, f"{path_dataset}annotations_train.json", f"{path_dataset}images")
# Cargar la instancia de COCO como un objeto Dataset
dataset_name = "piscinas_entrenamiento"
dataset = DatasetCatalog.get(dataset_name)
# Obtener metadatos del dataset
metadata = MetadataCatalog.get(dataset_name)
# Declarar el DataLoader
data_loader_train = DataLoader(dataset, batch_size=paramaters["batch_size"], shuffle=True, collate_fn=collate_fn)

# Registrar instancia de COCO
register_coco_instances("piscinas_test", {}, f"{path_dataset}annotations_test.json", f"{path_dataset}images")
# Cargar la instancia de COCO como un objeto Dataset
dataset_name = "piscinas_test"
dataset = DatasetCatalog.get(dataset_name)
# Obtener metadatos del dataset
metadata = MetadataCatalog.get(dataset_name)
# Declarar el DataLoader
data_loader_test = DataLoader(dataset, batch_size=paramaters["batch_size"], collate_fn=collate_fn_test)

# Declarar el dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Mover el modelo y el DataLoader al dispositivo
model.to(device)
train_loss = train(model, optimizer, data_loader_train, device, epochs = paramaters["epochs"])

Downloading: "https://github.com/facebookresearch/detr/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 135MB/s]
Downloading: "https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth" to /root/.cache/t

WARNING [04/17 09:32:01 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/17 09:32:01 d2.data.datasets.coco]: Loaded 582 images in COCO format from /content/drive/MyDrive/TFG/dataset_700x700/annotations_train.json
WARNING [04/17 09:32:02 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/17 09:32:02 d2.data.datasets.coco]: Loaded 72 images in COCO format from /content/drive/MyDrive/TFG/dataset_700x700/annotations_test.json
No congelando: model.bbox_embed.layers.0.weight
No congelando: model.bbox_embed.layers.0.bias
No congelando: model.bbox_embed.layers.1.weight
No congelando: model.bbox_embed.layers.1.bias
No congelando: model.bbox_embed.layers.2.weight
No congelando: model.bbox_embed.layers.2.bias
No congelando: model.query_embed.weight

Epoch: 0 



  0%|          | 0/146 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
  1%|          | 1/146 [00:08<21:40,  8.97s/it]

 - loss: 1.3931304216384888


  1%|▏         | 2/146 [00:10<11:02,  4.60s/it]

...

 69%|██████▉   | 101/146 [00:43<00:19,  2.29it/s]

 - loss: 1.2655067443847656


 70%|██████▉   | 102/146 [00:43<00:19,  2.28it/s]

 - loss: 1.3519866466522217


 71%|███████   | 103/146 [00:44<00:20,  2.14it/s]

In [ ]:
plt.plot(train_loss)
print(paramaters)

In [ ]:
for images, targets in data_loader_test:
  images = [image.to(device) for image in images]
  targets = [{k: v for k, v in t.items()} for t in targets]
  output = model(images)
  for n in range(len(images)):
    fig, ax = plt.subplots()
    image = images[n].cpu().numpy()
    image = np.moveaxis(image, 0, -1)
    for bounding_box, target in zip(output[0][n], output[1][n]):
    # for bounding_box in targets[0]["boxes"]:
      if target > 0.6:
        x, y, w, h = bounding_box.cpu().detach().numpy()
        x, y, w, h = x*new_size[0], y*new_size[1], w*new_size[0], h*new_size[1]
        bb = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(bb)
    ax.imshow(image)
    plt.show()

In [ ]:
output[1][0]

In [ ]:
torch.save(model, f"{path_dataset}model.pt")